<a href="https://colab.research.google.com/github/mems250005048-wq/GDG/blob/main/GDG_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import yfinance as yf
import pandas as pd

# Download historical data
ticker = "AAPL"
df = yf.download(ticker, start="2018-01-01", end="2024-01-01")

# Clean columns
df.reset_index(inplace=True)
df.rename(columns={
    "Date": "date",
    "Open": "open",
    "High": "high",
    "Low": "low",
    "Close": "close",
    "Volume": "volume"
}, inplace=True)

# Save CSV
df.to_csv("AAPL_OHLCV.csv", index=False)

print("AAPL OHLCV CSV created successfully")


/tmp/ipython-input-3320495629.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2018-01-01", end="2024-01-01")
[*********************100%***********************]  1 of 1 completed

AAPL OHLCV CSV created successfully
